In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
sns.set_style('darkgrid')
mpl.rcParams['figure.figsize'] = [18,10]

In [3]:
non_numeric = ['BMI_class', 'Height_class', 'Gender', 'Component', 'Branch']

def load_ansur(cols_to_drop):
    df_m = pd.read_csv('data/ANSUR_II_MALE.csv')
    df_f = pd.read_csv('data/ANSUR_II_FEMALE.csv')
    ansur_df = pd.concat([df_m, df_f], axis=0)

    X = ansur_df.drop(non_numeric, axis=1)
    y = ansur_df['Gender']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    return X, y, X_train, X_test, y_train, y_test

In [4]:
X, y, X_train, X_test, y_train, y_test = load_ansur(non_numeric)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)

### Creating a logistic regression model

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression()
lr.fit(X_train_std, y_train)

X_test_std = scaler.transform(X_test)

In [6]:
y_pred = lr.predict(X_test_std)
print(accuracy_score(y_test, y_pred))

1.0


### Inspecting the feature coefficients 

In [7]:
print(lr.coef_[0][:10])

[ 0.15888498  0.1240153  -0.05343749 -0.26243949 -0.10276858 -0.22050059
  0.16498785  0.75903248  0.7069799  -0.80607276]


In [8]:
coef_dict = dict(zip(X.columns, abs(lr.coef_[0])))

{k: v for i, (k, v) in enumerate(coef_dict.items()) if i < 10}

{'abdominalextensiondepthsitting': 0.15888498239167154,
 'acromialheight': 0.12401530236140436,
 'acromionradialelength': 0.053437488637074135,
 'anklecircumference': 0.2624394940034043,
 'axillaheight': 0.10276857930507866,
 'balloffootcircumference': 0.22050059095729263,
 'balloffootlength': 0.16498785159111393,
 'biacromialbreadth': 0.7590324796397779,
 'bicepscircumferenceflexed': 0.7069799010987571,
 'bicristalbreadth': 0.8060727560128965}

In [9]:
low_coef = {k: v for k, v in coef_dict.items() if v < .401}

cols = [k for k, v in low_coef.items()]

In [10]:
X.drop(cols, axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

lr.fit(scaler.fit_transform(X_train), y_train)

print(accuracy_score(y_test, lr.predict(scaler.transform(X_test))))

1.0


In [11]:
X, y, X_train, X_test, y_train, y_test = load_ansur(non_numeric)

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [12]:
from sklearn.feature_selection import RFE

rfe = RFE(estimator=LogisticRegression(), n_features_to_select=5, verbose=0)
rfe.fit(X_train_std, y_train)

RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                 fit_intercept=True, intercept_scaling=1,
                                 l1_ratio=None, max_iter=100,
                                 multi_class='auto', n_jobs=None, penalty='l2',
                                 random_state=None, solver='lbfgs', tol=0.0001,
                                 verbose=0, warm_start=False),
    n_features_to_select=5, step=1, verbose=0)

In [13]:
X.columns[rfe.support_]

Index(['biacromialbreadth', 'chestheight', 'hipbreadthsitting',
       'neckcircumference', 'tenthribheight'],
      dtype='object')

In [14]:
#print(dict(zip(X.columns, rfe.ranking_)))

In [15]:
print(accuracy_score(y_test, rfe.predict(X_test_std)))

0.9972542559033498


In [16]:
def load_pima(cols_to_drop):
    df = pd.read_csv('data/PimaIndians.csv')

    X = df.drop(cols_to_drop, axis=1)
    y = df['test']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    return X, y, X_train, X_test, y_train, y_test

X, y, X_train, X_test, y_train, y_test = load_pima('test')

scaler = StandardScaler()
lr = LogisticRegression()

In [17]:
# Fit the scaler on the training features and transform these in one go
X_train_std = scaler.fit_transform(X_train)

# Fit the logistic regression model on the scaled training data
lr.fit(X_train_std, y_train)

# Scale the test features
X_test_std = scaler.transform(X_test)

# Predict diabetes presence on the scaled test set
y_pred = lr.predict(X_test_std)

# Prints accuracy metrics and feature coefficients
print("{0:.1%} accuracy on test set.".format(accuracy_score(y_test, y_pred))) 
print(dict(zip(X.columns, abs(lr.coef_[0]).round(2))))

79.7% accuracy on test set.
{'pregnant': 0.24, 'glucose': 0.99, 'diastolic': 0.0, 'triceps': 0.01, 'insulin': 0.09, 'bmi': 0.57, 'family': 0.27, 'age': 0.33}


In [18]:
X, y, X_train, X_test, y_train, y_test = load_pima('test')

X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

# Create the RFE with a LogisticRegression estimator and 3 features to select
rfe = RFE(estimator=LogisticRegression(), n_features_to_select=3, verbose=1)

# Fits the eliminator to the data
rfe.fit(X_train_std, y_train)

# Print the features and their ranking (high = dropped early on)
print(dict(zip(X.columns, rfe.ranking_)))

# Print the features that are not eliminated
print(X.columns[rfe.support_])

# Calculates the test set accuracy
acc = accuracy_score(y_test, rfe.predict(X_test_std))
print("{0:.1%} accuracy on test set.".format(acc)) 

Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
{'pregnant': 3, 'glucose': 1, 'diastolic': 6, 'triceps': 4, 'insulin': 5, 'bmi': 1, 'family': 2, 'age': 1}
Index(['glucose', 'bmi', 'age'], dtype='object')
76.3% accuracy on test set.


In [19]:
X, y, X_train, X_test, y_train, y_test = load_ansur(non_numeric)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()

rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [21]:
print(accuracy_score(y_test, rf.predict(X_test)))

0.9945085118066996


In [22]:
print(rf.feature_importances_)

[0.00100486 0.00075385 0.00053242 0.00099681 0.00409187 0.01593118
 0.01522615 0.07493944 0.00207385 0.0068397  0.0076777  0.01922996
 0.00068126 0.00287844 0.00202835 0.0123815  0.0034858  0.00089305
 0.00148194 0.00172395 0.00184134 0.01723966 0.00112562 0.00113515
 0.00613624 0.02405235 0.00043086 0.00752699 0.00067926 0.00093025
 0.00090266 0.00276022 0.00066783 0.01380866 0.00502869 0.01295004
 0.01093212 0.04021021 0.00304139 0.00147237 0.00066569 0.04099455
 0.0647785  0.00081298 0.00099272 0.00075448 0.00079872 0.04949011
 0.00063824 0.01564411 0.02377723 0.00058118 0.00051539 0.00291452
 0.01586242 0.00066268 0.00064404 0.00158206 0.01032052 0.00451017
 0.00463721 0.14017949 0.08938459 0.00092123 0.0003629  0.00734205
 0.00182728 0.03110931 0.00446956 0.00109293 0.00425768 0.01915128
 0.00060966 0.00123584 0.00598705 0.00096531 0.00749515 0.00157464
 0.00071017 0.00076445 0.00059776 0.00073952 0.00125929 0.01968767
 0.00139982 0.00182944 0.00109968 0.0003709  0.00068359 0.0706

In [23]:
mask = rf.feature_importances_ > 0.03
print(mask)

[False False False False False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False  True False False False  True  True False False False False  True
 False False False False False False False False False False False False
 False  True  True False False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False  True False False False False]


In [24]:
X_reduced = X_train.loc[:, mask]
print(X_reduced.columns)

Index(['biacromialbreadth', 'forearmcircumferenceflexed', 'handbreadth',
       'handcircumference', 'heelanklecircumference', 'neckcircumference',
       'neckcircumferencebase', 'shouldercircumference', 'wristcircumference'],
      dtype='object')


### RFE with random forests

In [25]:
from sklearn.feature_selection import RFE

rfe = RFE(estimator=RandomForestClassifier(), 
          n_features_to_select=6, step=10, 
            verbose=1)

rfe.fit(X_train, y_train)

Fitting estimator with 94 features.
Fitting estimator with 84 features.
Fitting estimator with 74 features.
Fitting estimator with 64 features.
Fitting estimator with 54 features.
Fitting estimator with 44 features.
Fitting estimator with 34 features.
Fitting estimator with 24 features.
Fitting estimator with 14 features.


RFE(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                     class_weight=None, criterion='gini',
                                     max_depth=None, max_features='auto',
                                     max_leaf_nodes=None, max_samples=None,
                                     min_impurity_decrease=0.0,
                                     min_impurity_split=None,
                                     min_samples_leaf=1, min_samples_split=2,
                                     min_weight_fraction_leaf=0.0,
                                     n_estimators=100, n_jobs=None,
                                     oob_score=False, random_state=None,
                                     verbose=0, warm_start=False),
    n_features_to_select=6, step=10, verbose=1)

In [26]:
print(X_train.columns[rfe.support_])

Index(['biacromialbreadth', 'handcircumference', 'hipbreadthsitting',
       'neckcircumference', 'neckcircumferencebase', 'shouldercircumference'],
      dtype='object')


In [27]:
X, y, X_train, X_test, y_train, y_test = load_pima('test')

In [28]:
# Fit the random forest model to the training data
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

# Calculate the accuracy
acc = accuracy_score(y_test, rf.predict(X_test))

# Print the importances per feature
print(dict(zip(X_train.columns, rf.feature_importances_.round(2))))

# Print accuracy
print("{0:.1%} accuracy on test set.".format(acc))

{'pregnant': 0.1, 'glucose': 0.21, 'diastolic': 0.07, 'triceps': 0.1, 'insulin': 0.17, 'bmi': 0.1, 'family': 0.11, 'age': 0.14}
73.7% accuracy on test set.


In [29]:
# Create a mask for features importances above the threshold
mask = rf.feature_importances_ > 0.15

# Prints out the mask
print(mask)

[False  True False False  True False False False]


In [30]:
# Create a mask for features importances above the threshold
mask = rf.feature_importances_ > 0.15

# Apply the mask to the feature dataset X
reduced_X = X_train.loc[:, mask]

# prints out the selected column names
print(reduced_X.columns)

Index(['glucose', 'insulin'], dtype='object')


In [31]:
# Wrap the feature eliminator around the random forest model
rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=2, verbose=1)

In [32]:
# Fit the model to the training data
rfe.fit(X_train, y_train)

Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.


RFE(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                     class_weight=None, criterion='gini',
                                     max_depth=None, max_features='auto',
                                     max_leaf_nodes=None, max_samples=None,
                                     min_impurity_decrease=0.0,
                                     min_impurity_split=None,
                                     min_samples_leaf=1, min_samples_split=2,
                                     min_weight_fraction_leaf=0.0,
                                     n_estimators=100, n_jobs=None,
                                     oob_score=False, random_state=None,
                                     verbose=0, warm_start=False),
    n_features_to_select=2, step=1, verbose=1)

In [33]:
!jupyter nbconvert --to html week_1.ipynb

[NbConvertApp] WARNING | pattern 'week_1.ipynb' matched no files
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--debug
    set log level to logging.DEBUG (maximize logging output)
--generate-config
    generate default config file
-y
    Answer yes to any questions instead of prompting.
--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--stdin
    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'
--stdout
    Write notebook output to stdout ins

In [34]:
!../gitbsh > /dev/null 2>&1